In [31]:
%load_ext autoreload
%autoreload 2

import keras
import math
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from Jlayers import SimpleLayer
from utilsSimpleConv2D import*
from SpectralLayer import Spectral
from spectralconvolutions import *
from tensorflow.keras.layers import Dense,Layer
from typing import Tuple,List,Any,Dict
from tensorflow.python.keras import activations, initializers, regularizers, constraints

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
## premiere approche
N=6
M=6
F=3
S=1
filters=1
parameters_one={ "use_lambda_out":False,
                 "use_lambda_in":False,
                 "trainable_SM_kernel":True,
                 "use_bias":False,
                 "kernel_initializer":"glorot_uniform"
           }

In [45]:
layer=(SpectralConv2D_one(filters=1, kernel_size=3, strides=1, padding="SAME", activation=None,**parameters_one))

In [46]:
img=np.random.normal(0, 1,size=(1,N,M,1))
print(f"Image d'entrée: {img}")

Image d'entrée: [[[[-0.11058789]
   [ 0.5610843 ]
   [-0.25001861]
   [ 1.44179384]
   [-1.46574531]
   [-0.96255567]]

  [[-1.64356712]
   [-0.05281451]
   [ 1.27462668]
   [-0.24590598]
   [ 1.62998698]
   [ 0.09410845]]

  [[-0.26185412]
   [ 1.0116733 ]
   [ 0.66329297]
   [-1.14879754]
   [ 0.49632538]
   [-0.08384485]]

  [[ 0.35109525]
   [-0.61492068]
   [-1.20410103]
   [ 0.90783434]
   [ 1.09438419]
   [-1.97700282]]

  [[-1.33666141]
   [ 0.30207893]
   [ 0.24910929]
   [-0.69951647]
   [-1.03863907]
   [ 0.83405488]]

  [[ 1.00158613]
   [ 0.15135926]
   [ 1.26026933]
   [ 1.04381343]
   [-0.41573379]
   [-0.57825289]]]]


In [47]:
print(f"Convolution Matricielle:\n {layer(img)[0,:,:,0]}")

Convolution Matricielle:
 [[-1.0559856   0.06059347  0.16832924  1.7113345   1.8092008   0.62979704]
 [ 0.89008266  0.11357552 -0.6743766   0.13599592 -1.3802739   1.3624614 ]
 [-0.48875654 -0.80133885  0.25771928  0.08762383 -0.7002952  -1.2507904 ]
 [-0.46102482  0.39577952 -0.83835787 -2.2301846   0.993119    0.4116437 ]
 [ 0.87961817  0.90239227  1.960807    1.869839   -0.654708   -1.4559288 ]
 [-0.27319962  0.4546846  -0.67768115  0.23483384  0.84510857  0.54957926]]


In [49]:
omega=layer.get_omega()

In [51]:
print(f"Convolution Classique:\n {convolution(img=img[0,:,:,0],kernel=omega[0])}")

Convolution Classique:
 [[-1.0559856   0.06059343  0.16832925  1.7113345   1.8092008   0.62979704]
 [ 0.8900827   0.11357552 -0.6743766   0.13599592 -1.380274    1.3624614 ]
 [-0.48875648 -0.801339    0.25771928  0.08762378 -0.7002952  -1.2507902 ]
 [-0.46102485  0.39577955 -0.83835787 -2.2301848   0.9931191   0.41164368]
 [ 0.8796181   0.9023922   1.960807    1.8698388  -0.65470797 -1.4559289 ]
 [-0.27319962  0.45468462 -0.6776812   0.23483385  0.8451085   0.54957926]]


In [2]:
## Seconde Approche

In [59]:
N=6
M=6
F=3
S=3
filters=1
index=indices_phi(filters=filters,N=N,M=M,F=F,S=S)

In [60]:
out_shape1:int=math.floor((N-F)/S)+1
out_shape2:int=math.floor((M-F)/S)+1
output_lenght:int=out_shape1*out_shape2
noyau=tf.constant(np.ones((filters, F*F)),dtype="float32")
print(f"noyau: {noyau}")

noyau: [[1. 1. 1. 1. 1. 1. 1. 1. 1.]]


In [61]:
kernel=tf.repeat(noyau, repeats=output_lenght, axis=0, name=None)
kernel=tf.reshape(kernel,shape=(-1,filters*output_lenght*F*F))
kernel=tf.sparse.SparseTensor(
indices=index, values=kernel[0], dense_shape=(filters,output_lenght,N*M)
)
kernel=tf.sparse.to_dense(kernel)
print(f"Matrice de convolution M: {kernel}")

Matrice de convolution M: [[[1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0.
   0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.
   1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1.]]]


In [62]:
img=np.random.normal(0, 1,size=(N,M))
print(f"Image d'entrée: {img}")

Image d'entrée: [[-0.22246216 -2.09627529 -0.36918137  0.1195452   0.32177229  0.65963848]
 [-0.46130394 -0.58163001 -0.11297976 -1.43243222  0.14071547 -1.50724429]
 [ 0.03488263  0.15866136  0.03607143 -0.99991941 -1.59708492  0.0809257 ]
 [-1.14374806  0.16731095 -0.47600588  0.63772628  0.00901906  1.50442779]
 [ 0.13341463  0.14854315 -0.40912516 -0.3888402   1.14209035  0.74235159]
 [-1.22165522 -1.83637194  0.10667412  0.35587417 -0.45208858  1.17346321]]


In [63]:
noyau_conv=np.ones((F,F),dtype="float32")

In [64]:
print(f"Convolution Classique:\n {convolution(img=img,kernel=noyau_conv)}")

Convolution Classique:
 [[-3.3616714  -3.8438325  -4.4729533  -1.3325604  -1.6980051  -0.38511807]
 [-3.1681273  -3.614217   -5.27814    -3.8934934  -4.2140837  -1.9012773 ]
 [-1.8258271  -2.3787413  -2.6031973  -3.79489    -3.1638665  -1.3692412 ]
 [-0.5009354  -1.349995   -1.1255774  -2.0460684   1.1306963   1.8817296 ]
 [-3.7525065  -4.5309634  -1.6942145   0.52532417  4.724024    4.1192636 ]
 [-2.7760694  -3.0785205  -2.0232458   0.35458472  2.5728505   2.6058166 ]]


In [65]:
inputs=tf.constant(img,dtype="float32")
inputs=tf.reshape(inputs, shape=(N*M,1))
print(f"Convolution Matricielle:\n {tf.reshape(tf.matmul(a=kernel,b=inputs),shape=(out_shape1,out_shape2))}")

Convolution Matricielle:
 [[-3.6142173 -4.2140837]
 [-4.5309634  4.724024 ]]
